On Order Fleet inventory data extract cleaning process

Set up daily download from ePlan emailed each weekday as .xls, connect to PowerQuery

In [ ]:
    Source = Excel.Workbook(File.Contents("C:\Users\vwarren1\OneDrive - CBRE, Inc\Attachments\FleetInventoryOnOrder.xlsx"), null, true),

Filter Rows, Expand Data, Promote Headers

In [ ]:
Details_Sheet = Source{[Item="Details",Kind="Sheet"]}[Data],
    #"Promoted Headers" = Table.PromoteHeaders(Details_Sheet, [PromoteAllScalars=true]),

Changed data types for the following fields to text in order to Join with other data sources: Client Account Code, Year, AcctMethod Code, Client Unit Number, Make, Base Model, Vehicle Type, VIN, Fuel Type, Driver County, Driver City, Driver State, Driver Zip, Owner, Vehicle Telematics Serial #, Vehicle Contractor #, Vehicle GPS Supplier, Vehicle GPS Model#

In [ ]:
#"Changed Type" = Table.TransformColumnTypes(#"Promoted Headers",{{"Client Account Code", type text}, {"Year", type text}, {"AcctMethod Code", type text}, {"Client Unit Number", type text}, {"ePlan Unit Number", Int64.Type}, {"Make", type text}, {"Base Model", type text}, {"Vehicle Type", type text}, {"VIN", type text}, {"Fuel Type", type text}, {"Months In Service", Int64.Type}, {"Driver County", type text}, {"Driver City", type text}, {"Driver State", type text}, {"Driver Zip", type text}, {"Owner", type text}, {"Vehicle Telematics Serial #", type text}, {"Vehicle Contractor #", type text}, {"Vehicle GPS Supplier", type text}, {"Vehicle GPS Model#", type text}}),

Filtered out records unneeded for data analysis, including MDHD Trucks, Equipment, Golf Carts, Latin American and Canadian Vehicles, and null records

In [ ]:
#"Filtered out NonUS" = Table.SelectRows(#"Changed Type", each ([Owner] <> "LEASEPLAN CANADA") and ([Driver State] <> "PR") and ([Driver State] <> "AB")and ([Driver State] <> "BC") and ([Driver State] <> "MB") and ([Driver State] <> "NB") and ([Driver State] <> "NL") and ([Driver State] <> "NT") and ([Driver State] <> "NS") and ([Driver State] <> "NU") and ([Driver State] <> "ON") and ([Driver State] <> "PE") and ([Driver State] <> "QC") and ([Driver State] <> "SK") and ([Driver State] <> "YT")
),
    #"Filtered out MHDR" = Table.SelectRows(#"Filtered out NonUS", each ([Driver State] <> null) and ([Base Model] <> "EL XD UTILITY TRUCK" and [Base Model] <> "F-650 GAS" and [Base Model] <> "F-750 DIESEL")),

Left join OnOrder with FuelType_GHG on 1) Year = year, 2) Make = make, and 3) Base Model = baseModel to clean dirty/null fields for visualization purposes and standardize descriptions based on DoT data

In [ ]:
#"Merged Queries wDoT" = Table.FuzzyNestedJoin(#"Filtered out MHDR", {"Year", "Make", "Base Model"}, FuelType_GHG, {"year", "make", "baseModel"}, "Model_VehType", JoinKind.LeftOuter, [IgnoreCase=true, IgnoreSpace=true]),


Expand joined columns from FuelType_GHG to existing table

In [ ]:
#"Expanded Model_VehType" = Table.ExpandTableColumn(#"Merged Queries wDoT", "Model_VehType", {"year", "make", "baseModel", "ghgScore", "VClass", "Vehicle Type", "Fuel Type"}, {"year.1", "make.1", "baseModel", "ghgScore", "VClass", "Vehicle Type.1", "Fuel Type.1"}),


Reorder columns for ease of reading, removed duplicates on VIN, removed original/dirty columns from OnOrder

In [ ]:

#"Reordered Columns" = Table.ReorderColumns(#"Expanded Model_VehType",{"year.1", "make.1", "baseModel", "ghgScore", "VClass", "Vehicle Type.1", "Fuel Type.1", "Year", "Make", "Base Model", "Vehicle Type", "VIN", "Fuel Type", "Months In Service", "Driver County", "Driver City", "Driver State", "Driver Zip", "Owner", "ePlan Unit Number", "Client Unit Number", "AcctMethod Code", "Client Account Code", "Vehicle Telematics Serial #", "Vehicle Contractor #", "Vehicle GPS Supplier", "Vehicle GPS Model#"}),
    #"Removed Duplicates" = Table.Distinct(#"Reordered Columns", {"VIN"}),
    #"Removed Columns" = Table.RemoveColumns(#"Removed Duplicates",{"Year", "Make", "Base Model", "Vehicle Type", "VClass", "Fuel Type"}),
    #"Renamed Columns" = Table.RenameColumns(#"Removed Columns",{{"year.1", "Year"}, {"make.1", "Make"}, {"baseModel", "Model"}, {"ghgScore", "GHG Score"}}),
    #"Renamed Columns1" = Table.RenameColumns(#"Renamed Columns",{{"Vehicle Type.1", "Vehicle Type"}, {"Fuel Type.1", "Fuel Type"}}),

Changed column GHG Score to type Int64


In [ ]:
#"Changed Type1" = Table.TransformColumnTypes(#"Renamed Columns1",{{"GHG Score", Int64.Type}})

In [ ]:
in
    #"Changed Type1"